In [6]:
%%file producer1.py

import json
import time
import logging
import socket
from datetime import datetime
import requests
import pandas as pd
from confluent_kafka import Producer

KAFKA_BROKER = 'broker:9092'
TRANSACTION_TOPIC = 'streaming'
LAG = 0.5

def create_producer():
    try:
        producer = Producer({
            "bootstrap.servers": KAFKA_BROKER,
            "client.id": socket.gethostname(),
            "enable.idempotence": True,
            "batch.size": 64000,
            "linger.ms": 10,
            "acks": "all",
            "retries": 5,
            "delivery.timeout.ms": 1000
        })
    except Exception as e:
        logging.exception("Nie mogę utworzyć producenta")
        producer = None
    return producer

def fetch_data():
    url = 'http://api.citybik.es/v2/networks/velib'
    response = requests.get(url)

    if response.status_code != 200:
        print("Request failed. Status code:", response.status_code)
        return None

    data = response.json()
    stations = data['network']['stations']
    
    # Ekstrahowanie detali
    station_data = []
    for station in stations:
        station_info = {
            'Station Name': station['name'],
            'Latitude': station['latitude'],
            'Longitude': station['longitude'],
            'Number of empty slots': station.get('empty_slots', 0),
            'Number of free bikes': station.get('free_bikes', 0),
            'Number of e-bikes': station['extra'].get('ebikes', 0),
            'Payment terminal': station['extra'].get('payment-terminal', False),
            'Banking': station['extra'].get('banking', False),
            'Now renting': station['extra'].get('renting', 0),
            'Now returning': station['extra'].get('returning', 0),
            'Last updated': station.get('returning', 0)
        }
        station_data.append(station_info)

    return station_data

producer = create_producer()
if producer is not None:
    while True:
        data = fetch_data()
        if data:
            for station in data:
                current_time = datetime.utcnow().isoformat()
                station['current_time'] = current_time
                record = json.dumps(station).encode("utf-8")
                producer.produce(topic=TRANSACTION_TOPIC, value=record)
                print(f"Sent: {record}")
                producer.flush()
                time.sleep(LAG)
        else:
            print("Failed to fetch data. Retrying...")
            time.sleep(60)


Overwriting producer1.py
